<a href="https://colab.research.google.com/github/tourihasi/Openstudio/blob/main/%E3%82%B9%E3%82%B1%E3%82%B8%E3%83%A5%E3%83%BC%E3%83%AB%E5%BC%95%E7%B6%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# 1. インポート ------------------------------------------------------------
import sys
import openstudio as osd

# 2. Colab判定と入出力ヘルパ ----------------------------------------------
def _in_colab() -> bool:
    return "google.colab" in sys.modules

def _pick_first_osm(upload_result: dict) -> str:
    cands = [n for n in upload_result.keys() if n.lower().endswith(".osm")]
    if not cands:
        raise RuntimeError("拡張子 .osm のファイルが見つかりません。")
    return cands[0]

# 3. ヘルパー関数（存在確認/インデックス）---------------------------------
def _exists_by_name(optional_getter, name: str) -> bool:
    try:
        opt = optional_getter(name)
        return opt.is_initialized()
    except Exception:
        return False

def _index_by_name(objs):
    d = {}
    for o in objs:
        try:
            d[o.nameString()] = o
        except Exception:
            pass
    return d

# 4. ScheduleTypeLimits の転記 --------------------------------------------
def clone_schedule_type_limits(src_m: osd.model.Model, dst_m: osd.model.Model) -> int:
    dst_idx = _index_by_name(dst_m.getScheduleTypeLimitss())
    copied = 0
    for stl in src_m.getScheduleTypeLimitss():
        name = stl.nameString()
        if name in dst_idx:
            continue
        _ = stl.clone(dst_m)
        copied += 1
        print("[Copy] ScheduleTypeLimits:", name)
    print(f"[Info] ScheduleTypeLimits 転記: {copied} 件")
    return copied

# 5. 内部定義スケジュール＋ScheduleSet の転記 ------------------------------
def clone_internal_schedules(src_m: osd.model.Model, dst_m: osd.model.Model) -> None:
    """
    内部定義スケジュール（Ruleset/Constant/Compact/FixedInterval/Year/Week/Day）と
    DefaultScheduleSet を“定義だけ”一括転記（割当てはしない／Schedule:File除外）
    """
    # 5.1 TypeLimits 先行
    clone_schedule_type_limits(src_m, dst_m)

    # 5.2 スケジュール本体
    groups = [
        ("ScheduleRuleset",       src_m.getScheduleRulesets,       dst_m.getScheduleRulesetByName),
        ("ScheduleConstant",      src_m.getScheduleConstants,      dst_m.getScheduleConstantByName),
        ("ScheduleCompact",       src_m.getScheduleCompacts,       dst_m.getScheduleCompactByName),
        ("ScheduleFixedInterval", src_m.getScheduleFixedIntervals, dst_m.getScheduleFixedIntervalByName),
        ("ScheduleYear",          src_m.getScheduleYears,          dst_m.getScheduleYearByName),
        ("ScheduleWeek",          src_m.getScheduleWeeks,          dst_m.getScheduleWeekByName),
        ("ScheduleDay",           src_m.getScheduleDays,           dst_m.getScheduleDayByName),
    ]
    total = 0
    for label, src_getter, dst_by_name in groups:
        copied = 0
        for sch in src_getter():
            name = sch.nameString()
            if not _exists_by_name(dst_by_name, name):
                _ = sch.clone(dst_m)
                copied += 1
                total += 1
                print(f"[Copy] {label}:", name)
        print(f"[Info] {label} 転記: {copied} 件")
    print(f"[Done] 内部定義スケジュール 合計転記: {total} 件（TypeLimits除く）")

    # 5.3 DefaultScheduleSet
    dst_index = {s.nameString(): s for s in dst_m.getDefaultScheduleSets() if s.nameString()}
    copied_sets = 0
    for s in src_m.getDefaultScheduleSets():
        name = s.nameString()
        if not name or name in dst_index:
            continue
        _ = s.clone(dst_m)  # 参照先スケジュールは未存在なら同時に複製
        copied_sets += 1
        print("[Copy] DefaultScheduleSet:", name)
    print(f"[Done] スケジュールセット(DefaultScheduleSet) 転記: {copied_sets} 件")

# 6. 入出力フロー（Colab: アップロード/ダウンロード｜ローカル: パス指定）--------
if _in_colab():
    from google.colab import files
    print("転記基（source）OSM を選択してください")
    up_src = files.upload()
    src_path = _pick_first_osm(up_src)

    print("転記先（target）OSM を選択してください")
    up_dst = files.upload()
    dst_path = _pick_first_osm(up_dst)

    src_m = osd.model.Model.load(osd.path(src_path)).get()
    dst_m = osd.model.Model.load(osd.path(dst_path)).get()

    clone_internal_schedules(src_m, dst_m)

    out_name = "target_schedules_only.osm"
    dst_m.save(osd.path(out_name), True)
    print("Saved:", out_name)
    files.download(out_name)
else:
    # ローカル/サーバ運用の場合はここでパスを指定してください
    src_path = "source.osm"   # ← 転記基
    dst_path = "target.osm"   # ← 転記先
    src_m_opt = osd.model.Model.load(osd.path(src_path))
    dst_m_opt = osd.model.Model.load(osd.path(dst_path))
    assert src_m_opt.is_initialized(), f"{src_path} の読み込みに失敗"
    assert dst_m_opt.is_initialized(), f"{dst_path} の読み込みに失敗"
    src_m = src_m_opt.get()
    dst_m = dst_m_opt.get()

    clone_internal_schedules(src_m, dst_m)

    out_name = "target_schedules_only.osm"
    dst_m.save(osd.path(out_name), True)
    print("Saved:", out_name)


転記基（source）OSM を選択してください


Saving 下妻_空調.osm to 下妻_空調.osm
転記先（target）OSM を選択してください


Saving 90度回転.osm to 90度回転.osm
[Copy] ScheduleTypeLimits: OnOff hvac_library
[Copy] ScheduleTypeLimits: Fractional
[Copy] ScheduleTypeLimits: Integer
[Copy] ScheduleTypeLimits: Temperature 13
[Copy] ScheduleTypeLimits: Dimensionless 4
[Copy] ScheduleTypeLimits: Dimensionless
[Copy] ScheduleTypeLimits: Dimensionless 2
[Copy] ScheduleTypeLimits: Temperature 11
[Copy] ScheduleTypeLimits: ActivityLevel
[Copy] ScheduleTypeLimits: Temperature
[Copy] ScheduleTypeLimits: RotationsPerMinute
[Copy] ScheduleTypeLimits: Temperature 4
[Copy] ScheduleTypeLimits: Integer 1
[Copy] ScheduleTypeLimits: LinearPowerDensity
[Copy] ScheduleTypeLimits: Dimensionless 3
[Copy] ScheduleTypeLimits: Capacity
[Copy] ScheduleTypeLimits: ClothingInsulation
[Copy] ScheduleTypeLimits: VolumetricFlowRate
[Copy] ScheduleTypeLimits: ControlMode
[Copy] ScheduleTypeLimits: DeltaTemperature
[Copy] ScheduleTypeLimits: Dimensionless 1
[Copy] ScheduleTypeLimits: Fraction 1
[Copy] ScheduleTypeLimits: Temperature 12
[Copy] Schedu

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>